# MarketData 数据查看器

本notebook用于展示MarketData类中所有fetcher获取的实际数据内容，帮助快速了解每个接口的功能和数据结构。

## 📊 MarketData包含的数据类型

- **龙虎榜数据** - 大额交易明细
- **融资融券数据** - 融资融券余额统计
- **北向资金数据** - 外资持股情况
- **上交所数据** - 上海证券交易所市场统计
- **深交所数据** - 深圳证券交易所市场统计
- **地区成交数据** - 按地区分类的交易统计
- **行业成交数据** - 按行业分类的交易统计

## 1. 导入库并初始化

In [1]:
import sys
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 获取项目根目录
current_dir = os.path.abspath('.')
project_root = os.path.dirname(current_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 导入MarketData
from china_stock_data import MarketData

# 设置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_rows', 10)

print("✅ 导入成功")
print(f"项目根目录: {project_root}")

✅ 导入成功
项目根目录: /Users/juzhongsun/Codes/projects/china_stock_data


## 2. 创建MarketData实例

In [2]:
# 创建MarketData实例
market = MarketData(date='202408', symbol='当月')

print(f"MarketData实例创建成功！")
print(f"配置日期: {market.date}")
print(f"可用fetchers: {len(market.fetchers)}个")

# 获取所有fetcher信息
unique_fetchers = {}
for name, fetcher in market.fetchers.items():
    fetcher_id = id(fetcher)
    if fetcher_id not in unique_fetchers:
        unique_fetchers[fetcher_id] = {
            'instance': fetcher,
            'names': [name],
            'class_name': fetcher.__class__.__name__
        }
    else:
        unique_fetchers[fetcher_id]['names'].append(name)

print(f"唯一fetchers: {len(unique_fetchers)}个")

MarketData实例创建成功！
配置日期: 20240801
可用fetchers: 17个
唯一fetchers: 8个


## 3. 数据展示函数

In [3]:
def display_fetcher_data(fetcher_name, description, market_instance):
    """
    展示指定fetcher的数据内容和结构
    """
    print(f"\n{'='*80}")
    print(f"📊 {description}")
    print(f"🔗 访问名称: {fetcher_name}")
    print(f"{'='*80}")
    
    try:
        # 获取数据
        data = market_instance.get_data(fetcher_name)
        
        if data.empty:
            print("❌ 数据为空")
            return
        
        # 基本信息
        print(f"📈 数据形状: {data.shape[0]}行 x {data.shape[1]}列")
        print(f"📋 列名: {list(data.columns)}")
        
        # 数据类型
        print(f"\n🔢 数据类型:")
        for col, dtype in data.dtypes.items():
            print(f"   {col}: {dtype}")
        
        # 数据预览
        print(f"\n📄 数据预览 (前5行):")
        print(data.head())
        
        if len(data) > 10:
            print(f"\n📄 数据预览 (后5行):")
            print(data.tail())
        
        # 数值列的统计信息
        numeric_cols = data.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            print(f"\n📊 数值列统计:")
            print(data[numeric_cols].describe())
        
        print(f"\n✅ 数据获取成功")
        
    except Exception as e:
        print(f"❌ 获取数据时出错: {str(e)}")

## 4. 龙虎榜数据

In [4]:
display_fetcher_data('lhb', '龙虎榜数据 - 显示大额交易的股票及营业部信息', market)


📊 龙虎榜数据 - 显示大额交易的股票及营业部信息
🔗 访问名称: lhb
📈 数据形状: 637行 x 21列
📋 列名: ['序号', '代码', '名称', '上榜日', '解读', '收盘价', '涨跌幅', '龙虎榜净买额', '龙虎榜买入额', '龙虎榜卖出额', '龙虎榜成交额', '市场总成交额', '净买额占总成交比', '成交额占总成交比', '换手率', '流通市值', '上榜原因', '上榜后1日', '上榜后2日', '上榜后5日', '上榜后10日']

🔢 数据类型:
   序号: int64
   代码: int64
   名称: object
   上榜日: object
   解读: object
   收盘价: float64
   涨跌幅: float64
   龙虎榜净买额: float64
   龙虎榜买入额: float64
   龙虎榜卖出额: float64
   龙虎榜成交额: float64
   市场总成交额: int64
   净买额占总成交比: float64
   成交额占总成交比: float64
   换手率: float64
   流通市值: float64
   上榜原因: object
   上榜后1日: float64
   上榜后2日: float64
   上榜后5日: float64
   上榜后10日: float64

📄 数据预览 (前5行):
   序号  代码    名称         上榜日                解读    收盘价     涨跌幅       龙虎榜净买额  \
0   1  21   深科技  2023-04-06  1家机构买入，成功率46.03%  20.70  9.9894  85829823.97   
1   2  21   深科技  2023-04-03  实力游资买入，成功率46.29%  18.39  9.9880   8153093.45   
2   3  21   深科技  2023-04-03  2家机构买入，成功率25.51%  18.39  9.9880  91704516.31   
3   4  32  深桑达A  2023-04-06    买一主买，成功率46.37%  39.28  9.9972   884

## 5. 融资融券数据

In [5]:
display_fetcher_data('margin_financing', '融资融券数据 - 融资融券余额统计', market)


📊 融资融券数据 - 融资融券余额统计
🔗 访问名称: margin_financing
📈 数据形状: 2001行 x 8列
📋 列名: ['信用交易日期', '融资余额', '融资买入额', '融券余量', '融券余量金额', '融券卖出量', '融资融券余额', '融券余额']

🔢 数据类型:
   信用交易日期: float64
   融资余额: float64
   融资买入额: float64
   融券余量: float64
   融券余量金额: float64
   融券卖出量: float64
   融资融券余额: float64
   融券余额: float64

📄 数据预览 (前5行):
       信用交易日期          融资余额         融资买入额          融券余量        融券余量金额  \
0  20230922.0  8.016024e+11  3.092686e+10  7.214267e+09  5.677832e+10   
1  20230921.0  8.021216e+11  2.146390e+10  7.218682e+09  5.580829e+10   
2  20230920.0  8.016527e+11  2.121474e+10  7.274828e+09  5.643445e+10   
3  20230919.0  8.015889e+11  2.372691e+10  7.354550e+09  5.741973e+10   
4  20230918.0  8.017627e+11  2.777818e+10  7.404446e+09  5.788819e+10   

         融券卖出量        融资融券余额  融券余额  
0  735951864.0  8.583808e+11   NaN  
1  628790638.0  8.579299e+11   NaN  
2  639706111.0  8.580871e+11   NaN  
3  756797457.0  8.590087e+11   NaN  
4  727715296.0  8.596509e+11   NaN  

📄 数据预览 (后5行):
          信用

## 6. 北向资金持股数据

In [6]:
display_fetcher_data('northbound_holdings', '北向资金持股数据 - 外资通过沪深港通持有的A股', market)


📊 北向资金持股数据 - 外资通过沪深港通持有的A股
🔗 访问名称: northbound_holdings
📈 数据形状: 1336行 x 16列
📋 列名: ['序号', '代码', '名称', '今日收盘价', '今日涨跌幅', '今日持股-股数', '今日持股-市值', '今日持股-占流通股比', '今日持股-占总股本比', '5日增持估计-股数', '5日增持估计-市值', '5日增持估计-市值增幅', '5日增持估计-占流通股比', '5日增持估计-占总股本比', '所属板块', '日期']

🔢 数据类型:
   序号: int64
   代码: int64
   名称: object
   今日收盘价: float64
   今日涨跌幅: float64
   今日持股-股数: float64
   今日持股-市值: float64
   今日持股-占流通股比: float64
   今日持股-占总股本比: float64
   5日增持估计-股数: float64
   5日增持估计-市值: float64
   5日增持估计-市值增幅: float64
   5日增持估计-占流通股比: float64
   5日增持估计-占总股本比: float64
   所属板块: object
   日期: object

📄 数据预览 (前5行):
   序号      代码    名称  今日收盘价  今日涨跌幅    今日持股-股数     今日持股-市值  今日持股-占流通股比  \
0   1  600900  长江电力  29.62  -0.60  193829.58  5741232.04        8.07   
1   2  601318  中国平安  41.61   0.70   58873.40  2449722.19        5.47   
2   3  600050  中国联通   4.73   4.42  115290.52   545324.17        3.72   
3   4  601127   赛力斯  81.84   5.59    2459.57   201291.53        1.63   
4   5  600030  中信证券  19.10  -0.26   47838.25   913

## 7. 上海证券交易所数据

In [7]:
display_fetcher_data('sse_summary', '上交所市场总貌 - 上海证券交易所股票统计', market)


📊 上交所市场总貌 - 上海证券交易所股票统计
🔗 访问名称: sse_summary
📈 数据形状: 8行 x 4列
📋 列名: ['项目', '股票', '主板', '科创板']

🔢 数据类型:
   项目: object
   股票: float64
   主板: float64
   科创板: float64

📄 数据预览 (前5行):
      项目         股票         主板       科创板
0   流通股本   47486.96   45600.48   1886.47
1    总市值  615369.35  517811.79  97557.55
2  平均市盈率      15.64      14.07     68.46
3   上市公司    2287.00    1698.00    589.00
4   上市股票    2325.00    1736.00    589.00

📊 数值列统计:
                 股票            主板           科创板
count  8.000000e+00  8.000000e+00  8.000000e+00
mean   2.693606e+06  2.670336e+06  2.554644e+06
std    7.099052e+06  7.107077e+06  7.150492e+06
min    1.564000e+01  1.407000e+01  6.846000e+01
25%    2.315500e+03  1.726500e+03  5.890000e+02
50%    4.896978e+04  4.686073e+04  2.109050e+03
75%    5.888335e+05  5.020400e+05  8.679353e+04
max    2.025093e+07  2.025093e+07  2.025093e+07

✅ 数据获取成功


In [8]:
display_fetcher_data('sse_deal_daily', '上交所每日成交概况 - 每日交易统计数据', market)


📊 上交所每日成交概况 - 每日交易统计数据
🔗 访问名称: sse_deal_daily
📈 数据形状: 9行 x 6列
📋 列名: ['单日情况', '股票', '主板A', '主板B', '科创板', '股票回购']

🔢 数据类型:
   单日情况: object
   股票: float64
   主板A: float64
   主板B: float64
   科创板: float64
   股票回购: float64

📄 数据预览 (前5行):
    单日情况           股票          主板A          主板B          科创板         股票回购
0   市价总值    456375.26    406925.21       853.72     48596.33         0.00
1  平均市盈率        12.52        11.83         6.56        32.56          NaN
2    成交量       348.30       329.55         0.41        18.34         0.09
3   成交金额      3535.82      3030.14         1.23       504.45         0.99
4   报告日期  20240801.00  20240801.00  20240801.00  20240801.00  20240801.00

📊 数值列统计:
                 股票           主板A           主板B           科创板          股票回购
count  9.000000e+00  9.000000e+00  9.000000e+00  9.000000e+00  8.000000e+00
mean   2.347651e+06  2.338070e+06  2.249149e+06  2.258392e+06  2.530100e+06
std    6.712639e+06  6.715732e+06  6.746869e+06  6.743428e+06  7.156204e+06
min    7.

## 8. 深圳证券交易所数据

In [9]:
display_fetcher_data('szse_summary', '深交所市场总貌 - 深圳证券交易所证券类别统计', market)


📊 深交所市场总貌 - 深圳证券交易所证券类别统计
🔗 访问名称: szse_summary
📈 数据形状: 14行 x 5列
📋 列名: ['证券类别', '数量', '成交金额', '总市值', '流通市值']

🔢 数据类型:
   证券类别: object
   数量: int64
   成交金额: float64
   总市值: float64
   流通市值: float64

📄 数据预览 (前5行):
    证券类别    数量          成交金额           总市值          流通市值
0     股票  2885  4.301566e+11  2.682510e+13  2.281776e+13
1   主板A股  1496  2.309774e+11  1.734211e+13  1.561392e+13
2   主板B股    41  7.236322e+07  4.666270e+10  4.652017e+10
3  创业板A股  1348  1.991069e+11  9.436329e+12  7.157320e+12
4     基金   705  2.349177e+10  7.174885e+11  7.017962e+11

📄 数据预览 (后5行):
    证券类别     数量          成交金额           总市值          流通市值
9     债券  14006  2.625878e+11           NaN           NaN
10  债券现券  13373  4.563887e+10  7.396094e+13  2.787560e+12
11  债券回购     16  2.163858e+11           NaN           NaN
12   ABS    617  5.631240e+08  4.178347e+11  4.178347e+11
13    期权    396  5.871861e+08           NaN           NaN

📊 数值列统计:
                 数量          成交金额           总市值          流通市值
count     1

In [10]:
display_fetcher_data('szse_area_summary', '深交所地区成交排序 - 按地区统计的交易数据', market)


📊 深交所地区成交排序 - 按地区统计的交易数据
🔗 访问名称: szse_area_summary
📈 数据形状: 34行 x 7列
📋 列名: ['序号', '地区', '总交易额', '占市场', '股票交易额', '基金交易额', '债券交易额']

🔢 数据类型:
   序号: int64
   地区: object
   总交易额: float64
   占市场: float64
   股票交易额: float64
   基金交易额: float64
   债券交易额: float64

📄 数据预览 (前5行):
   序号  地区          总交易额     占市场         股票交易额         基金交易额         债券交易额
0   1  上海  4.800766e+12  17.976  2.223904e+12  1.894037e+11  2.386916e+12
1   2  深圳  3.461262e+12  12.960  1.602554e+12  1.623486e+11  1.696359e+12
2   3  北京  2.758256e+12  10.328  1.265754e+12  1.337782e+11  1.358182e+12
3   4  浙江  2.125194e+12   7.958  1.437788e+12  5.497876e+10  6.324268e+11
4   5  江苏  1.988666e+12   7.446  1.096425e+12  7.705387e+10  8.151874e+11

📄 数据预览 (后5行):
    序号   地区          总交易额    占市场         股票交易额         基金交易额         债券交易额
29  30   新疆  7.375254e+10  0.276  5.124575e+10  1.102387e+09  2.140441e+10
30  31  内蒙古  7.088729e+10  0.265  3.997399e+10  9.643077e+08  2.994900e+10
31  32   甘肃  5.856915e+10  0.219  4.302041e+10  

In [11]:
display_fetcher_data('szse_sector_summary', '深交所行业成交统计 - 按行业分类的交易数据', market)


📊 深交所行业成交统计 - 按行业分类的交易数据
🔗 访问名称: szse_sector_summary
📈 数据形状: 20行 x 9列
📋 列名: ['项目名称', '项目名称-英文', '交易天数', '成交金额-人民币元', '成交金额-占总计', '成交股数-股数', '成交股数-占总计', '成交笔数-笔', '成交笔数-占总计']

🔢 数据类型:
   项目名称: object
   项目名称-英文: object
   交易天数: int64
   成交金额-人民币元: int64
   成交金额-占总计: float64
   成交股数-股数: int64
   成交股数-占总计: float64
   成交笔数-笔: int64
   成交笔数-占总计: float64

📄 数据预览 (前5行):
   项目名称        项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计       成交股数-股数  成交股数-占总计  \
0    合计          Total    22  7415637047186    100.00  830332351047    100.00   
1  农林牧渔    Agriculture    22    68301507023      0.92    8340424844      1.00   
2   采矿业         Mining    22    98326994141      1.33   11742501083      1.41   
3   制造业  Manufacturing    22  5169258663118     69.71  495605330423     59.69   
4  水电煤气      Utilities    22    85173028841      1.15   16746580179      2.02   

      成交笔数-笔  成交笔数-占总计  
0  829761431    100.00  
1    7156991      0.86  
2   11609200      1.40  
3  566569084     68.28  
4   13903988      1.6

## 9. 多名称访问演示

## 10. 完整总览

In [12]:
print("📊 MarketData完整功能总览")
print("="*80)

print(f"\n🏗️ 系统信息:")
print(f"   配置日期: {market.date}")
print(f"   YYYYMM格式: {market.date_yyyymm}")
print(f"   YYYYMMDD格式: {market.date_yyyymmdd}")
print(f"   符号参数: {market.symbol}")

print(f"\n📈 数据获取能力:")
working_count = 0
total_data_rows = 0

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    primary_name = info['names'][0]
    class_name = info['class_name']
    all_names = ' | '.join(info['names'])
    
    try:
        data = market.get_data(primary_name)
        if not data.empty:
            working_count += 1
            total_data_rows += len(data)
            status = f"✅ {data.shape[0]}行 x {data.shape[1]}列"
        else:
            status = "⚠️ 空数据"
    except Exception as e:
        status = f"❌ 错误"
    
    print(f"   {i}. {class_name}: {status}")
    print(f"      访问名称: {all_names}")

print(f"\n🎯 统计结果:")
print(f"   正常工作的fetchers: {working_count}/{len(unique_fetchers)}")
print(f"   总数据行数: {total_data_rows:,}行")
print(f"   访问名称总数: {len(market.fetchers)}个")
print(f"   成功率: {working_count/len(unique_fetchers)*100:.1f}%")

print(f"\n💡 使用建议:")
print(f"   • 使用业务名称访问更直观: market.lhb, market.sse_summary")
print(f"   • 使用akshare名称保持兼容性: market.stock_lhb_detail_em")
print(f"   • 通过get_data()方法统一访问: market.get_data('lhb')")
print(f"   • 所有数据都支持pandas DataFrame操作")

📊 MarketData完整功能总览

🏗️ 系统信息:
   配置日期: 20240801
   YYYYMM格式: 202408
   YYYYMMDD格式: 20240801
   符号参数: 当月

📈 数据获取能力:
   1. LHBFetcher: ✅ 637行 x 21列
      访问名称: lhb | stock_lhb_detail_em
   2. MarginFinancingFetcher: ✅ 2001行 x 8列
      访问名称: margin_financing | stock_margin_sse | stock_margin_szse
   3. NorthboundHoldingsFetcher: ✅ 1336行 x 16列
      访问名称: northbound_holdings | stock_hsgt_hold_stock_em
   4. SSESummaryFetcher: ✅ 8行 x 4列
      访问名称: sse_summary | stock_sse_summary
   5. SZSESummaryFetcher: ✅ 14行 x 5列
      访问名称: szse_summary | stock_szse_summary
   6. SZSEAreaSummaryFetcher: ✅ 34行 x 7列
      访问名称: szse_area_summary | stock_szse_area_summary
   7. SZSESectorSummaryFetcher: ✅ 20行 x 9列
      访问名称: szse_sector_summary | stock_szse_sector_summary
   8. SSEDealDailyFetcher: ✅ 9行 x 6列
      访问名称: sse_deal_daily | stock_sse_deal_daily

🎯 统计结果:
   正常工作的fetchers: 8/8
   总数据行数: 4,059行
   访问名称总数: 17个
   成功率: 100.0%

💡 使用建议:
   • 使用业务名称访问更直观: market.lhb, market.sse_summary
   • 使用akshare名